![JohnSnowLabs](https://sparknlp.org/assets/images/logo.png)

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/JohnSnowLabs/spark-nlp/blob/master/examples/python/transformers/HuggingFace%20in%20Spark%20NLP%20-%20ALBERT%20-%20S3.ipynb)

## Import ALBERT models from HuggingFace 🤗  into Spark NLP 🚀 with S3

Let's keep in mind a few things before we start 😊

- In Spark NLP you can configure the store repository to save these models. Starting in Spark NLP 5.1.0, we can define a local file system, or a distributed file system (DBFS) like HDFS or Databricks FS. You can also set an S3 bucket, Azure Storage or GCP Storage.
- You can import models for ALBERT from HuggingFace but they have to be compatible with `TensorFlow` and they have to be in `Fill Mask` category. Meaning, you cannot use ALBERT models trained/fine-tuned on a specific task such as token/sequence classification.

In this notebook, we are going to see the steps required to use an external S3 repository. To do this, we need to configure the spark session with the required settings for Spark NLP and Spark ML.

### Spark NLP Settings

Spark NLP requires the following configuration:
- S3 Region: We need the region to upload a file on your S3 bucket. This is defined in the config `spark.jsl.settings.aws.region`

### Spark ML Settings

This configuration will depend on your S3 bucket and AWS configuration. In this notebook a connection through **Temporary Security Credentials** is showcased. **Please contact your administrator to choose the right setup, as well as, the required keys/tokens.**

Spark ML requires the following configuration to save a model in S3 using *Temporary Security Credentials*:

1. Authenticating with S3: This is needed to interact with external S3 buckets, and it will require an access key, a secret key, and a session token. Define the values in these configs:

- `spark.hadoop.fs.s3a.access.key`
- `spark.hadoop.fs.s3a.secret.key`
- `spark.hadoop.fs.s3a.session.token`
2. Credential Provider: You need to define the Hadoop provider that will handle this connection. Since in this notebook, *Temporary Security Credentials* is used we need to use the provider `TemporaryAWSCredentialsProvider` from `hadoop-aws` package, and set it up in the config below:

- `spark.hadoop.fs.s3a.aws.credentials.provider`
3. AWS packages: S3A depends upon two JARs, alongside `hadoop-common` and its dependencies, which are `hadoop-aws` and `aws-java-sdk` packages. So, you will need to either add these dependencies in your application or to your spark session. Since we are using a notebook, we will add these packages while creating the spark session in the following config:

- `spark.jars.packages`
4. AWS File System: Defining S3AFileSystem it's also required for interacting S3 with AWS SDK. Define the value in this config:

- `spark.hadoop.fs.s3a.impl`

In [ ]:
print("Enter your AWS Access Key:")
MY_ACCESS_KEY = input()

In [ ]:
print("Enter your AWS Secret Key:")
MY_SECRET_KEY = input()

In [ ]:
print("Enter your AWS Session Key:")
MY_SESSION_KEY = input()

In [ ]:
print("Enter your AWS Region:")
MY_AWS_REGION

In [ ]:
#S3 Storage configuration
s3_params = {
    "spark.hadoop.fs.s3a.aws.credentials.provider": "org.apache.hadoop.fs.s3a.TemporaryAWSCredentialsProvider",
    "spark.hadoop.fs.s3a.impl": "org.apache.hadoop.fs.s3a.S3AFileSystem",
    "spark.hadoop.fs.s3a.path.style.access": "true",
    "spark.hadoop.fs.s3a.access.key": MY_ACCESS_KEY,
    "spark.hadoop.fs.s3a.secret.key": MY_SECRET_KEY,
    "spark.hadoop.fs.s3a.session.token": MY_SESSION_KEY,
    "spark.jsl.settings.aws.region": MY_AWS_REGION,
    "spark.jars.packages": "org.apache.hadoop:hadoop-aws:3.3.1,com.amazonaws:aws-java-sdk:1.11.901"
}

## Export and Save HuggingFace model

- Let's install `HuggingFace` and `TensorFlow`. You don't need `TensorFlow` to be installed for Spark NLP, however, we need it to load and save models from HuggingFace.
- We lock TensorFlow on `2.11.0` version and Transformers on `4.25.1`. This doesn't mean it won't work with the future releases, but we wanted you to know which versions have been tested successfully.
- AlbertTokenizer requires the `SentencePiece` library, so we install that as well

In [ ]:
!pip install -q transformers==4.25.1 tensorflow==2.11.0 sentencepiece

- HuggingFace comes with a native `saved_model` feature inside `save_pretrained` function for TensorFlow based models. We will use that to save it as TF `SavedModel`.
- We'll use [albert-base-v2](https://huggingface.co/albert-base-v2) model from HuggingFace as an example
- In addition to `TFAlbertModel` we also need to save the `AlbertTokenizer`. This is the same for every model, these are assets needed for tokenization inside Spark NLP.
- Since `albert-base-v2` model is PyTorch we will use `from_pt=True` param to convert it to TensorFlow

In [ ]:
from transformers import AlbertTokenizer, TFAlbertModel
import tensorflow as tf

# albert-base-v2
MODEL_NAME = 'albert-base-v2'

AlbertTokenizer.from_pretrained(MODEL_NAME, return_tensors="pt").save_pretrained("./{}_tokenizer".format(MODEL_NAME))

# just in case if there is no TF/Keras file provided in the model
# we can just use `from_pt` and convert PyTorch to TensorFlow
try:
  print('try downloading TF weights')
  model = TFAlbertModel.from_pretrained(MODEL_NAME)
except:
  print('try downloading PyTorch weights')
  model = TFAlbertModel.from_pretrained(MODEL_NAME, from_pt=True)

# Define TF Signature
@tf.function(
  input_signature=[
      {
          "input_ids": tf.TensorSpec((None, None), tf.int32, name="input_ids"),
          "attention_mask": tf.TensorSpec((None, None), tf.int32, name="attention_mask"),
          "token_type_ids": tf.TensorSpec((None, None), tf.int32, name="token_type_ids"),
      }
  ]
)
def serving_fn(input):
    return model(input)

model.save_pretrained("./{}".format(MODEL_NAME), saved_model=True, signatures={"serving_default": serving_fn})

try downloading TF weights


Some layers from the model checkpoint at albert-base-v2 were not used when initializing TFAlbertModel: ['predictions']
- This IS expected if you are initializing TFAlbertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFAlbertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFAlbertModel were initialized from the model checkpoint at albert-base-v2.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFAlbertModel for predictions without further training.


Let's have a look inside these two directories and see what we are dealing with:

In [ ]:
!ls -l {MODEL_NAME}

total 45684
-rw-r--r-- 1 root root      792 Jun 19 22:26 config.json
drwxr-xr-x 3 root root     4096 Jun 19 22:26 saved_model
-rw-r--r-- 1 root root 46771352 Jun 19 22:26 tf_model.h5


In [ ]:
!ls -l {MODEL_NAME}/saved_model/1

total 10048
drwxr-xr-x 2 root root     4096 Jun 19 22:26 assets
-rw-r--r-- 1 root root       54 Jun 19 22:26 fingerprint.pb
-rw-r--r-- 1 root root    24303 Jun 19 22:26 keras_metadata.pb
-rw-r--r-- 1 root root 10248326 Jun 19 22:26 saved_model.pb
drwxr-xr-x 2 root root     4096 Jun 19 22:26 variables


In [ ]:
!ls -l {MODEL_NAME}_tokenizer

total 752
-rw-r--r-- 1 root root    286 Jun 19 22:25 special_tokens_map.json
-rw-r--r-- 1 root root 760289 Jun 19 22:25 spiece.model
-rw-r--r-- 1 root root    577 Jun 19 22:25 tokenizer_config.json


- as you can see, we need the SavedModel from `saved_model/1/` path
- we also be needing `spiece.model` file from the tokenizer
- all we need is to copy `spiece.model` file into `saved_model/1/assets` which Spark NLP will look for

In [ ]:
# let's copy spiece.model file to saved_model/1/assets
!cp {MODEL_NAME}_tokenizer/spiece.model {MODEL_NAME}/saved_model/1/assets

Now upload the model in your S3 repository. You just need to copy all the contents inside `{MODEL_NAME}/saved_model/1` to an S3 bucket. For example in `s3://my_s3_path/saved_model/albert-base/`

In [ ]:
print("Enter your S3 path to store these models:")
S3_MODEL_PATH = input()

## Import and Save ALBERT in Spark NLP


- Let's install and setup Spark NLP in Google Colab
- This part is pretty easy via our simple script

In [ ]:
! pip install -q pyspark==3.2.3 spark-nlp==4.4.3

Let's start Spark with Spark NLP included via our simple `start()` function

In [ ]:
spark = sparknlp.start(params=s3_params)

print("Apache Spark version: {}".format(spark.version))

- Let's use `loadSavedModel` functon in `AlbertEmbeddings` which allows us to load TensorFlow model in SavedModel format
- Most params can be set later when you are loading this model in `AlbertEmbeddings` in runtime, so don't worry what you are setting them now
- `loadSavedModel` accepts two params, first is the path to the TF SavedModel. The second is the SparkSession that is `spark` variable we previously started via `sparknlp.start()`
- `setStorageRef` is very important. When you are training a task like NER or any Text Classification, we use this reference to bound the trained model to this specific embeddings so you won't load a different embeddings by mistake and see terrible results 😊
- It's up to you what you put in `setStorageRef` but it cannot be changed later on. We usually use the name of the model to be clear, but you can get creative if you want!
- The `dimension` param is is purely cosmetic and won't change anything. It's mostly for you to know later via `.getDimension` what is the dimension of your model. So set this accordingly.
- NOTE: `loadSavedModel` accepts local paths in addition to distributed file systems such as `HDFS`, `S3`, `DBFS`, etc. This feature was introduced in Spark NLP 4.2.2 release. Keep in mind the best and recommended way to move/share/reuse Spark NLP models is to use `write.save` so you can use `.load()` from any file systems natively.


In [ ]:
from sparknlp.annotator import *

albert = AlbertEmbeddings.loadSavedModel(
     '{}/saved_model/1'.format(MODEL_NAME),
     spark
 )\
 .setInputCols(["sentence",'token'])\
 .setOutputCol("embeddings")\
 .setCaseSensitive(False)\
 .setDimension(768)\
 .setStorageRef('albert_base_uncased')

- Let's save it on disk so it is easier to be moved around and also be used later via `.load` function

In [ ]:
albert.write().overwrite().save("./{}_spark_nlp".format(MODEL_NAME))

Let's clean up stuff we don't need anymore

In [ ]:
!rm -rf {MODEL_NAME}_tokenizer {MODEL_NAME}

Awesome 😎  !

This is your ALBERT model from HuggingFace 🤗 loaded and saved by Spark NLP 🚀

In [ ]:
! ls -l {MODEL_NAME}_spark_nlp

total 56376
-rw-r--r-- 1 root root   760289 Jun 19 22:27 albert_spp
-rw-r--r-- 1 root root 56957780 Jun 19 22:27 albert_tensorflow
drwxr-xr-x 3 root root     4096 Jun 19 22:27 fields
drwxr-xr-x 2 root root     4096 Jun 19 22:27 metadata


Now let's see how we can use it on other machines, clusters, or any place you wish to use your new and shiny RoBERTa model 😊

In [ ]:
albert_loaded = AlbertEmbeddings.load("./{}_spark_nlp".format(MODEL_NAME))\
  .setInputCols(["sentence",'token'])\
  .setOutputCol("embeddings")\
  .setCaseSensitive(False)

In [ ]:
albert_loaded.getStorageRef()

'albert_base_uncased'

That's it! You can now go wild and use hundreds of ALBERT models from HuggingFace 🤗 in Spark NLP 🚀


In [ ]:
albert = AlbertEmbeddings.loadSavedModel(S3_MODEL_PATH, spark)\
 .setInputCols(["sentence",'token'])\
 .setOutputCol("embeddings")\
 .setCaseSensitive(False)\
 .setDimension(768)\
 .setStorageRef('albert_base_uncased')